# try 1

## import

In [1]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *
# import rec_util
pd.options.display.float_format = '{:.2f}'.format
import pandas as pd

## 파일 로드

In [1]:
raw_tr = pd.read_json("./raw_data/train.json")
raw_te = pd.read_json("./raw_data/val.json")

In [10]:
len(raw_te[['tags','songs','id','plylst_title']][raw_te['songs'].apply(len) == 0]) / len(raw_te)

0.1902672170323702

In [6]:
raw_te[['tags','songs','id','plylst_title']][raw_te['tags'].apply(len) == 0]

,tags,songs,id,plylst_title
0,[],"[373313, 151080, 275346, 696876, 165237, 52593...",118598,
1,[],[],131447,앨리스테이블
2,[],"[529437, 516103, 360067, 705713, 226062, 37089...",51464,
3,[],"[589668, 21711, 570151, 320043, 13930, 599327,...",45144,
4,[],"[672718, 121924, 102694, 683657, 201558, 38511...",79929,
...,...,...,...,...
23003,[],"[474446, 111476, 241237, 70500, 577313, 145058...",127349,
23006,[],"[486938, 154124, 70969, 596414, 243733, 557848...",92067,
23012,[],"[625875, 464051, 11657, 236393, 358186, 213435...",77438,
23013,[],"[161094, 665833, 688145, 432735, 439938, 12665...",36231,


In [3]:
raw_tr

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


## 태그별 빈도 확인

In [4]:
ret = []
for tag in raw_tr.tags.tolist():
    ret += tag
from collections import Counter
r = dict(Counter(ret))

In [5]:
r = sorted(r.items(), key=lambda x: -x[1])
r[:5]

[('기분전환', 16465), ('감성', 11417), ('휴식', 11215), ('발라드', 10796), ('잔잔한', 10218)]

In [6]:
top_tags = [x[0] for x in r[:1000]]

# 태그와 노래 목록 만으로 나머지 예측하기

In [7]:
train_songs = raw_tr['songs'].tolist()
test_songs = raw_te['songs'].apply(lambda x : [float(i)for i in x]).tolist()
train_tags = raw_tr['tags'].tolist()
test_tags = raw_te['tags'].tolist()
test_ids = raw_te['id'].tolist()

### train 데이터 list to list 로 만들기

In [8]:
from itertools import groupby
tr = []
iid_to_idx = {}
idx = 0

for i, l in enumerate(train_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

In [9]:
idx = 0
n_items = 615142
tag_to_idx = {} 
for i, tags in enumerate(train_tags):
    for tag in tags:
        if tag not in tag_to_idx:
            tag_to_idx[tag] = n_items + idx
            idx += 1
    tr[i].extend([tag_to_idx[x] for x in tags])
n_items = len(iid_to_idx)
n_tags = len(tag_to_idx)

### test 데이터 list to list 로 만들기

In [10]:
te = []

idx = 0
for i, l in enumerate(test_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)
idx = 0
for i, tags in enumerate(test_tags):
    ret = []
    for tag in tags:
        if tag not in tag_to_idx:
            continue
        ret.append(tag)
    te[i].extend([tag_to_idx[x] for x in ret])

# 데이터 쓰까

In [11]:
tr = shuffle(tr)

### 노래 및 태그 인덱싱 하기

In [12]:
idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}
idx_to_tag = {(x - n_items):y for(y,x) in tag_to_idx.items()}

# sparse matrix
- 방법에 대해 고민 해봐야 함

In [13]:
def boolean_indexing(v):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.zeros(mask.shape,dtype=int)
    out[mask] = np.concatenate(v)
    return out

In [14]:
lens = np.array([len(item) for item in tr])
lens.shape

(115071,)

In [15]:
mask = lens[:,None] > np.arange(lens.max())
mask.shape

(115071, 210)

In [16]:
out = np.zeros(mask.shape,dtype=int)
out.shape

(115071, 210)

In [17]:
tr_array = boolean_indexing(tr)

In [18]:
raw_tr

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [19]:
len(raw_tr.iloc[115070]['songs']) + len(raw_tr.iloc[115070]['tags'])

204

In [20]:
len(tr_array[115070])

210

In [21]:
tr_array.shape

(115071, 210)

In [28]:
te_array = boolean_indexing(te)

In [29]:
te_array.shape

(23015, 105)

In [30]:
tr_array = np.zeros([len(tr),len(max(tr,key = lambda x: len(x)))])
for i,j in enumerate(tr):
    tr_array[i][0:len(j)] = j

In [31]:
# 노래 곡수 + 태그의 최대 숫자가 240인데 이렇게 나오면 안되고, 'n_tags + n_items'인 115071 x 644302인 어레이가 나와야 함

In [32]:
np.array([len (i) for i in tr]).max()

210

In [33]:
n_tags + n_items

644302

## binary matrix 만들기
- https://stackoverflow.com/questions/50702988/creating-binary-matrix-of-each-unique-value-from-list-of-lists

- train list to list

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
tr_array = cv.fit_transform(tr)

# To transform to dense matrix
tr_array.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [23]:
len(raw_tr.iloc[0]['songs']) + len(raw_tr.iloc[0]['tags'])

20

In [24]:
tr_array.todense()[0].sum()

12

- test list to list

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
te_array = cv.fit_transform(te)

# To transform to dense matrix
te_array.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [26]:
te_array.shape

(23015, 130523)

In [64]:
# from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# configure
encoding_dim = 32
input_data = Input(shape=(644302,))


# layers
encoded = Dense(encoding_dim, activation='relu')(input_data)
decoded = Dense(644302, activation='sigmoid')(encoded)

# Models
autoencoder = Model(input_data, decoded) # autoencoder
encoder = Model(input_data, encoded) # encoder
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder


In [61]:
X = tr_array.todense()
X.shape

(115071, 644302)

In [65]:
# result viewer
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def recall(y_true, y_pred):
    y_true_yn = K.round(K.clip(y_true, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_true_yn * y_pred_yn)
    count_true_positive_false_negative = K.sum(y_true_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall

# train autoencoder
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=[rmse, recall])
autoencoder.fit(X, X,
                epochs=50,
                batch_size=256,
                shuffle=True,validation_split=0.2)

Train on 92056 samples, validate on 23015 samples
Epoch 1/50
  512/92056 [..............................] - ETA: 1:27:43 - loss: 0.6931 - rmse: 0.5000 - recall: 0.4957

KeyboardInterrupt: 

In [ ]:
# encoding result
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# classification train data
reducted_x_train = encoder.predict(x_train)
reducted_x_test = encoder.predict(x_test)

# train classifier
model=Sequential()
model.add(Dense(64, activation='relu', input_dim=encoding_dim))
model.add(layers.Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# final result
history = model.fit(reducted_x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)
performance_test = model.evaluate(reducted_x_test, y_test, batch_size=100)
print('Test Loss and Accuracy ->', performance_test)

# 모델링
- 1. sparse 행렬 작업
- 2. 두 행렬을 합치고
- 3. 모델 fit 시키기

In [27]:
tr_array

<115071x644302 sparse matrix of type '<class 'numpy.int64'>'
	with 5762202 stored elements in Compressed Sparse Row format>

In [28]:
te_array

<23015x130523 sparse matrix of type '<class 'numpy.int64'>'
	with 423707 stored elements in Compressed Sparse Row format>

In [29]:
tr_csr = csr_matrix(tr_array, (len(tr), n_tags + n_items))
te_csr = csr_matrix(te_array, (len(te), n_tags + n_items))

In [37]:
tr_csr

<115071x644302 sparse matrix of type '<class 'numpy.int64'>'
	with 5762202 stored elements in Compressed Sparse Row format>

In [35]:
import scipy.sparse
r = scipy.sparse.vstack([tr_csr, te_csr])
r = csr_matrix(r)

In [36]:
r

<138086x644302 sparse matrix of type '<class 'numpy.int64'>'
	with 6185909 stored elements in Compressed Sparse Row format>

In [41]:
raw_df = pd.concat([raw_tr, raw_te ])

In [42]:
from scipy.sparse import csr_matrix
like_cnt = csr_matrix(raw_df.like_cnt)
like_cnt

<1x138086 sparse matrix of type '<class 'numpy.int64'>'
	with 125718 stored elements in Compressed Sparse Row format>

In [43]:
like_cnt.T

<138086x1 sparse matrix of type '<class 'numpy.int64'>'
	with 125718 stored elements in Compressed Sparse Column format>

In [44]:
r

<138086x644302 sparse matrix of type '<class 'numpy.int64'>'
	with 6185909 stored elements in Compressed Sparse Row format>

In [45]:
r = scipy.sparse.hstack([r,like_cnt.T])
r

<138086x644303 sparse matrix of type '<class 'numpy.int64'>'
	with 6311627 stored elements in COOrdinate format>

In [ ]:
# ALS 말고 다른걸로 시도

In [28]:
als_model = ALS(factors=256, regularization=0.08)
als_model.fit(r.T * 15.0)

### 노래 예측 모댈과, 태그 예측모댈에 user factor와 item factor를 분배

In [29]:
item_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
item_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

In [50]:
item_model = LMF(use_gpu=False)
tag_model = LMF(use_gpu=False)
item_model.user_factors = LMF_model.user_factors
tag_model.user_factors = LMF_model.user_factors

In [52]:
item_model.item_factors = LMF_model.item_factors[:n_items]
tag_model.item_factors = LMF_model.item_factors[n_items:-1]

In [53]:
LMF_model.item_factors.shape

(644303, 258)

In [31]:
als_model.item_factors.shape

(644302, 256)

In [32]:
als_model.item_factors[n_items:]

array([[ 1.9774567e-01, -1.6457781e-01, -2.4069002e-02, ...,
         3.8423190e-01,  2.2601548e-01,  5.1538068e-01],
       [-2.3180166e-01, -4.1238788e-01,  1.8979147e-01, ...,
         2.5333425e-01,  1.1671105e-01,  6.6225190e-04],
       [-1.3898441e-01, -3.7492254e-01,  1.6532259e-01, ...,
         2.5847656e-01,  8.5519031e-02,  1.0789641e-02],
       ...,
       [ 6.0895982e-04,  1.2598315e-03,  1.5657897e-04, ...,
         6.7041896e-04,  8.2416815e-04,  2.6454786e-03],
       [ 6.7919539e-04, -7.3085364e-04,  1.4212606e-03, ...,
        -7.8746898e-04, -7.3015870e-04,  1.2854089e-03],
       [ 9.5603382e-04,  6.5275340e-04,  4.8686855e-04, ...,
        -7.2426465e-04, -1.5322662e-04,  3.2020046e-04]], dtype=float32)

In [56]:
item_rec_csr = tr_csr[:, :n_items]
tag_rec_csr = tr_csr[:, n_items:]

## recommend 과정

In [66]:
item_ret = []
tag_ret = []
from tqdm.auto import tqdm
for u in tqdm(range(te_csr.shape[0])):
    item_rec = item_model.recommend(u, item_rec_csr, N=100)
    item_rec = [idx_to_iid[x[0]] for x in item_rec]
    tag_rec = tag_model.recommend(u, tag_rec_csr, N=100)
    tag_rec = [idx_to_tag[x[0]] for x in tag_rec if x[0] in idx_to_tag]
    item_ret.append(item_rec)
    tag_ret.append(tag_rec)

In [35]:
tag_model.item_factors

array([[ 1.9774567e-01, -1.6457781e-01, -2.4069002e-02, ...,
         3.8423190e-01,  2.2601548e-01,  5.1538068e-01],
       [-2.3180166e-01, -4.1238788e-01,  1.8979147e-01, ...,
         2.5333425e-01,  1.1671105e-01,  6.6225190e-04],
       [-1.3898441e-01, -3.7492254e-01,  1.6532259e-01, ...,
         2.5847656e-01,  8.5519031e-02,  1.0789641e-02],
       ...,
       [ 6.0895982e-04,  1.2598315e-03,  1.5657897e-04, ...,
         6.7041896e-04,  8.2416815e-04,  2.6454786e-03],
       [ 6.7919539e-04, -7.3085364e-04,  1.4212606e-03, ...,
        -7.8746898e-04, -7.3015870e-04,  1.2854089e-03],
       [ 9.5603382e-04,  6.5275340e-04,  4.8686855e-04, ...,
        -7.2426465e-04, -1.5322662e-04,  3.2020046e-04]], dtype=float32)

In [26]:
tag_model.item_factors

array([[ 3.55746090e-01, -5.69548428e-01, -5.59754260e-02, ...,
         4.83457536e-01,  1.95368275e-01,  1.14752978e-01],
       [ 1.13077201e-01,  4.99642581e-01,  1.01401411e-01, ...,
         8.70525062e-01,  5.08861303e-01,  4.67959046e-01],
       [ 1.34080434e-02,  5.13347864e-01,  1.56607270e-01, ...,
         6.01837337e-01,  4.08071131e-01,  3.32719356e-01],
       ...,
       [-5.10524551e-04,  3.28546972e-04,  8.81741638e-04, ...,
         8.72330857e-04,  1.68862971e-04,  1.18788111e-03],
       [ 9.88858286e-04,  1.02399255e-03,  3.29685328e-03, ...,
         1.42479164e-03, -1.69632386e-03, -1.65981124e-03],
       [ 9.73270435e-05,  1.62653669e-04, -2.30579590e-05, ...,
         1.98975016e-04,  2.20649337e-04, -1.06205756e-04]], dtype=float32)

## 최종 json파일 추출

In [67]:
returnval = []
for _id, rec, tag_rec in zip(test_ids, item_ret, tag_ret):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [68]:
import json
with open('results.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))